# Boruta

In [52]:
## Import the required libraries
import pandas as pd
import numpy as np
import imblearn
from imblearn.pipeline import make_pipeline as make_pipeline_imbfinal 
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
#from sklearn import svm
from sklearn.metrics import *
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
df = pd.read_csv('ckd.csv')

In [36]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
drop_col_list=['RBC_normal','PCC_present','BA_present','CAD_yes','Ane_yes' , 'PC_normal' , 'Appet_good' ,
               'age' , 'pot' , 'wc' , 'PE_yes' , 'bp' , 'su' ]
x_train=df_train.iloc[:,:24]
x_train.drop(drop_col_list,axis=1,inplace=True)
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:24]
x_test.drop(drop_col_list,axis=1,inplace=True)
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [37]:
x_train_sc

array([[  1.33397733e+00,   1.58370532e+00,  -2.97910559e-01, ...,
         -9.64307503e-01,   1.32102159e+00,   1.33126903e+00],
       [  4.11721399e-01,  -6.74889702e-01,   1.19476398e+00, ...,
          7.80754115e-04,  -7.56989899e-01,   1.33126903e+00],
       [  4.11721399e-01,  -6.74889702e-01,  -4.05490706e-01, ...,
          7.80754115e-04,   1.32102159e+00,  -7.51162972e-01],
       ..., 
       [  1.33397733e+00,  -6.74889702e-01,  -4.45833262e-01, ...,
          3.50469731e-01,  -7.56989899e-01,  -7.51162972e-01],
       [  4.11721399e-01,  -6.74889702e-01,  -6.07203482e-01, ...,
          2.02382258e+00,  -7.56989899e-01,  -7.51162972e-01],
       [ -1.43279047e+00,  -6.74889702e-01,  -7.01336111e-01, ...,
          7.80754115e-04,  -7.56989899e-01,  -7.51162972e-01]])

In [38]:
x_test

,sg,al,bgr,bu,sc,sod,hemo,pcv,rc,HTN_yes,DM_yes
209,1.020,0.0,99.0,57.0,3.072454,138.0,11.500000,39.0,4.707435,0,0
280,1.020,0.0,93.0,33.0,0.900000,144.0,13.300000,52.0,5.200000,0,0
33,1.020,2.0,140.0,55.0,2.500000,138.0,10.100000,29.0,4.707435,1,0
210,1.015,4.0,255.0,132.0,12.800000,135.0,7.300000,20.0,3.900000,1,1
93,1.010,3.0,295.0,90.0,5.600000,140.0,9.200000,30.0,3.200000,1,1
84,1.010,3.0,76.0,186.0,15.000000,135.0,7.100000,22.0,2.100000,1,0
329,1.025,0.0,80.0,25.0,0.900000,146.0,14.100000,48.0,5.100000,0,0
94,1.010,0.0,93.0,66.0,1.600000,137.0,11.600000,36.0,3.900000,0,1
266,1.020,0.0,133.0,17.0,1.200000,135.0,13.200000,41.0,5.300000,0,0
126,1.015,0.0,144.0,125.0,4.000000,136.0,12.000000,37.0,4.500000,1,1


In [54]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train.iloc[:,:24]
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:24]
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

**Implementing Boruta**

In [43]:
import pandas as pd
#from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
X = x_train_sc
y = y_train

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels

rfc = RandomForestClassifier(n_estimators=50,random_state=0)

# define Boruta feature selection method
feat_selector_rf = BorutaPy(rfc, n_estimators='auto', verbose=2)

# find all relevant features
feat_selector_rf.fit(X, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	12
Tentative: 	8
Rejected: 	4
Iteration: 	9 / 100
Confirmed: 	12
Tentative: 	8
Rejected: 	4
Iteration: 	10 / 100
Confirmed: 	12
Tentative: 	8
Rejected: 	4

C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]



Iteration: 	11 / 100
Confirmed: 	12
Tentative: 	8
Rejected: 	4
Iteration: 	12 / 100
Confirmed: 	14
Tentative: 	6
Rejected: 	4
Iteration: 	13 / 100
Confirmed: 	14
Tentative: 	6
Rejected: 	4


C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	14 / 100
Confirmed: 	14
Tentative: 	6
Rejected: 	4
Iteration: 	15 / 100
Confirmed: 	14
Tentative: 	6
Rejected: 	4


...

C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	94 / 100
Confirmed: 	15
Tentative: 	2
Rejected: 	7
Iteration: 	95 / 100
Confirmed: 	15
Tentative: 	2
Rejected: 	7
Iteration: 	96 / 100
Confirmed: 	15
Tentative: 	2
Rejected: 	7
Iteration: 	97 / 100
Confirmed: 	15
Tentative: 	2
Rejected: 	7


C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]
C:\ProgramData\Anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	98 / 100
Confirmed: 	15
Tentative: 	2
Rejected: 	7
Iteration: 	99 / 100
Confirmed: 	15
Tentative: 	2
Rejected: 	7


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	15
Tentative: 	0
Rejected: 	7


BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=58, n_jobs=1,
            oob_score=False,
            random_state=<mtrand.RandomState object at 0x0000023A204B8E58>,
            verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x0000023A204B8E58>,
     two_step=True, verbose=2)

In [55]:
feat_selector_rf.support_

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False, False, False, False,
        True,  True, False,  True, False, False], dtype=bool)

Retriving important features after executing Boruta.

In [56]:
x_train.columns[feat_selector_rf.support_]

Index(['age', 'bp', 'sg', 'al', 'bgr', 'bu', 'sc', 'sod', 'hemo', 'pcv', 'wc',
       'rc', 'HTN_yes', 'DM_yes', 'Appet_good'],
      dtype='object')